great expectations with CSV 

In [1]:
import pandas as pd 
import great_expectations as gx

In [2]:
# GX 0.18.21+ Method

# Create context
context = gx.get_context()

# Create sample data
df = pd.read_csv(
    "https://raw.githubusercontent.com/great-expectations/gx_tutorials/main/data/yellow_tripdata_sample_2019-01.csv"
)

display(df)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-15 03:36:12,2019-01-15 03:42:19,1,1.00,1,N,230,48,1,6.5,0.5,0.5,1.95,0.0,0.3,9.75,NaN
1,1,2019-01-25 18:20:32,2019-01-25 18:26:55,1,0.80,1,N,112,112,1,6.0,1.0,0.5,1.55,0.0,0.3,9.35,0.0
2,1,2019-01-05 06:47:31,2019-01-05 06:52:19,1,1.10,1,N,107,4,2,6.0,0.0,0.5,0.00,0.0,0.3,6.80,NaN
3,1,2019-01-09 15:08:02,2019-01-09 15:20:17,1,2.50,1,N,143,158,1,11.0,0.0,0.5,3.00,0.0,0.3,14.80,NaN
4,1,2019-01-25 18:49:51,2019-01-25 18:56:44,1,0.80,1,N,246,90,1,6.5,1.0,0.5,1.65,0.0,0.3,9.95,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,2019-01-02 07:48:44,2019-01-02 08:00:13,6,1.07,1,N,50,161,2,8.5,0.0,0.5,0.00,0.0,0.3,9.30,NaN
9996,2,2019-01-16 19:06:45,2019-01-16 19:10:05,6,0.35,1,N,234,234,1,4.0,1.0,0.5,1.16,0.0,0.3,6.96,NaN
9997,2,2019-01-02 09:10:44,2019-01-02 09:36:46,6,4.12,1,N,50,236,1,20.0,0.0,0.5,6.24,0.0,0.3,27.04,NaN
9998,2,2019-01-03 13:28:36,2019-01-03 13:36:42,6,1.17,1,N,137,234,1,7.0,0.0,0.5,0.90,0.0,0.3,8.70,NaN


In [3]:
# Create data source using the official approach
data_source = context.sources.add_pandas('official_data_assistant_demo')

# Create asset
data_asset = data_source.add_dataframe_asset(name='official_dataframe')

# Build batch request
batch_request = data_asset.build_batch_request(dataframe=df)

# Get batch
batch_list = data_asset.get_batch_list_from_batch_request(batch_request)
batch = batch_list[0]

# Create expectation suite first
suite = context.add_expectation_suite('official_data_assistant_suite')

# Create validator using the official approach
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name='official_data_assistant_suite'
)

In [4]:
# Test Missingness Data Assistant
try:
    print('Testing Missingness Data Assistant...')
    missingness_result = context.assistants.missingness.run(
        validator=validator,
        exclude_column_names=['payment_type']
    )
    
    # The suite should be called 'missingness_suite'
    missingness_suite = missingness_result.get_expectation_suite(
        expectation_suite_name='missingness_suite'
    )
    
    print(f'✅ Missingness Data Assistant: {len(missingness_suite.expectations)} expectations')
    
    # Save the missingness expectation suite
    validator.expectation_suite = missingness_suite
    validator.save_expectation_suite(discard_failed_expectations=False)
    print('✅ Missingness Expectation Suite saved')

    # Build Data Docs for the missingness suite
    context.build_data_docs()
    print('✅ Data Docs built for missingness suite')

    # Try to open Data Docs for the missingness suite, but handle the case where the resource identifier is not found
    try:
        from great_expectations.data_context.types.resource_identifiers import ExpectationSuiteIdentifier
        resource_identifier = ExpectationSuiteIdentifier(missingness_suite.expectation_suite_name)
        context.open_data_docs(resource_identifier=resource_identifier)
        print('✅ Data Docs opened for missingness suite')
    except Exception as e_open:
        print(f'⚠️ Could not open Data Docs for missingness suite: {e_open}')
        print('You can manually open Data Docs from the index.html in your data_docs directory.')

    # Run a checkpoint for the missingness suite
    missingness_checkpoint = context.add_or_update_checkpoint(
        name='missingness_data_assistant_checkpoint',
        validator=validator,
    )
    missingness_checkpoint_result = missingness_checkpoint.run()
    print(f'✅ Missingness Checkpoint executed: {missingness_checkpoint_result["success"]}')

except Exception as e:
    print(f'Missingness Data Assistant error: {e}')

Testing Missingness Data Assistant...



Generating Expectations:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Profiling Dataset:         0%|          | 0/17 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/5 [00:00<?, ?it/s]

✅ Missingness Data Assistant: 17 expectations
✅ Missingness Expectation Suite saved
✅ Data Docs built for missingness suite
✅ Data Docs opened for missingness suite


Calculating Metrics:   0%|          | 0/88 [00:00<?, ?it/s]

✅ Missingness Checkpoint executed: True


In [6]:
# Test Onboarding Data Assistant
try:
    print('Testing Onboarding Data Assistant...')
    onboarding_result = context.assistants.onboarding.run(
        validator=validator,
        exclude_column_names=['payment_type']
    )
    
    onboarding_suite = onboarding_result.get_expectation_suite(
        expectation_suite_name='onboarding_suite'
    )
    
    print(f'✅ Onboarding Data Assistant: {len(onboarding_suite.expectations)} expectations')
    
    # Save the onboarding expectation suite
    validator.expectation_suite = onboarding_suite
    validator.save_expectation_suite(discard_failed_expectations=False)
    print('✅ Onboarding Expectation Suite saved')

    # Build Data Docs for the onboarding suite
    context.build_data_docs()
    print('✅ Data Docs built for onboarding suite')

    # Try to open Data Docs for the onboarding suite, but handle the case where the resource identifier is not found
    try:
        # Try using the ExpectationSuiteIdentifier instead of just the suite name
        from great_expectations.data_context.types.resource_identifiers import ExpectationSuiteIdentifier
        resource_identifier = ExpectationSuiteIdentifier(onboarding_suite.expectation_suite_name)
        context.open_data_docs(resource_identifier=resource_identifier)
        print('✅ Data Docs opened for onboarding suite')
    except Exception as e_open:
        print(f'⚠️ Could not open Data Docs for onboarding suite: {e_open}')
        print('You can manually open Data Docs from the index.html in your data_docs directory.')

    # Run a checkpoint for the onboarding suite
    onboarding_checkpoint = context.add_or_update_checkpoint(
        name='onboarding_data_assistant_checkpoint',
        validator=validator,
    )
    onboarding_checkpoint_result = onboarding_checkpoint.run()
    print(f'✅ Onboarding Checkpoint executed: {onboarding_checkpoint_result["success"]}')

except Exception as e:
    print(f'Onboarding Data Assistant error: {e}')

Testing Onboarding Data Assistant...



Generating Expectations:   0%|          | 0/8 [00:00<?, ?it/s]

Profiling Dataset:         0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

✅ Onboarding Data Assistant: 111 expectations
✅ Onboarding Expectation Suite saved
✅ Data Docs built for onboarding suite
✅ Data Docs opened for onboarding suite


Calculating Metrics:   0%|          | 0/257 [00:00<?, ?it/s]

✅ Onboarding Checkpoint executed: True


In [7]:
# Test Volume Data Assistant
try:
    print('Testing Volume Data Assistant...')
    volume_result = context.assistants.volume.run(
        validator=validator,
        exclude_column_names=['payment_type']
    )
    
    volume_suite = volume_result.get_expectation_suite(
        expectation_suite_name='volume_suite'
    )
    
    print(f'✅ Volume Data Assistant: {len(volume_suite.expectations)} expectations')
    
    # Save the volume expectation suite
    validator.expectation_suite = volume_suite
    validator.save_expectation_suite(discard_failed_expectations=False)
    print('✅ Volume Expectation Suite saved')

    # Build Data Docs for the volume suite
    context.build_data_docs()
    print('✅ Data Docs built for volume suite')

    # Try to open Data Docs for the volume suite, but handle the case where the resource identifier is not found
    try:
        from great_expectations.data_context.types.resource_identifiers import ExpectationSuiteIdentifier
        resource_identifier = ExpectationSuiteIdentifier(volume_suite.expectation_suite_name)
        context.open_data_docs(resource_identifier=resource_identifier)
        print('✅ Data Docs opened for volume suite')
    except Exception as e_open:
        print(f'⚠️ Could not open Data Docs for volume suite: {e_open}')
        print('You can manually open Data Docs from the index.html in your data_docs directory.')

    # Run a checkpoint for the volume suite
    volume_checkpoint = context.add_or_update_checkpoint(
        name='volume_data_assistant_checkpoint',
        validator=validator,
    )
    volume_checkpoint_result = volume_checkpoint.run()
    print(f'✅ Volume Checkpoint executed: {volume_checkpoint_result["success"]}')

except Exception as e:
    print(f'Volume Data Assistant error: {e}')

print('Method: context.assistants.{assistant_name}.run()')


Testing Volume Data Assistant...



Generating Expectations:   0%|          | 0/2 [00:00<?, ?it/s]

Profiling Dataset:         0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Volume Data Assistant: 8 expectations
✅ Volume Expectation Suite saved
✅ Data Docs built for volume suite
✅ Data Docs opened for volume suite


Calculating Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

✅ Volume Checkpoint executed: True
Method: context.assistants.{assistant_name}.run()


In [ ]:
# Open Data Docs in browser
print("\nOpening Data Docs in browser...")
context.open_data_docs()